# Solving a Complex Capacity Expansion Model with Stochastic Optimization

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

We demonstrate the application of stocastic optimization to solve the capacity expansion model illustrated in [Complex Capacity Expanion](07-Complex-Capacity-Expansion.ipynb).

In the `complex_expansion_data/` path, we provide several different sets of inputs using different sample time periods, for your use and experimentation, including 10 days (used as default below), 4 weeks, 8 weeks, 16 weeks, and 52 weeks (full 8760 hours).

Note that the open-source solver HiGHS may struggle to solve models with more than 10 days. Alter the `inputs_path` parameter below to select a different time series if desired.

# Package for plotting

In a separate julia window install `CairoMakie`
(i.e.
```{julia}
using Pkg
Pkg.add(["CairoMakie"])
```
) as it will take a moment to compile.

In [ ]:
using Pkg
Pkg.add("CairoMakie")

## LOAD PACKAGES

In [ ]:
# Ensure these packages are installed; if not, use the Pkg package and Pkg.add() function to install
using JuMP
# using Plots
using DataFrames, CSV
import Printf

# If do not have gurobi working, use this:
# using HiGHS
# optimizer = optimizer_with_attributes(HiGHS.Optimizer, "solver" => "ipm");

# If do have Gurobi, will be much faster
using Gurobi
optimizer = optimizer_with_attributes(Gurobi.Optimizer)

### Main stochastic model:


This is the main workhorse function for this lab and is more complicated than previous JuMP models seen so far.


As with any sort of complicated code, make a few passes, getting a sense of:
- the main indices
- the different types constraints we’ve discussed in class (e.g. storage, existing/new-built generators, ramping limits, etc).
- how stochastic programming modifies the structure (hint: look for the scenario indices Sc)
- where does the 'planning' half end and the 'operations' half begin

Pick one of the operational constraints and walk through it in detail, discussing each part, it's indices, and give a human-readable version of the constraint

Explain why the objective and `eOperationCosts` looks the way it does

In [ ]:
# Gives `load_inputs` and `scaling!` function 
include("../../benders_cems.jl")
function generate_stochastic_model(
    inputs::Dict{Symbol,Any},
    scenario_inputs::Vector{Dict{Symbol,Any}},
    scenario_weights::Vector{Float64};
    silent::Bool = false
)
    m = Model(optimizer)
    
    @assert length(scenario_inputs) == length(scenario_weights) "Length mismatch"
    Ns = length(scenario_inputs)
    Sc = 1:Ns

    # Shared sets and data (must be identical across scenarios)
    G          = inputs[:G]
    L          = inputs[:L]
    STOR       = inputs[:STOR]
    NEW        = inputs[:NEW]
    OLD        = inputs[:OLD]
    generators = inputs[:generators]
    lines      = inputs[:lines]

    # Validate common sets across scenarios
    T = scenario_inputs[1][:T]
    S = scenario_inputs[1][:S]
    Z = scenario_inputs[1][:Z]
    hours_per_period = scenario_inputs[1][:hours_per_period]

    for s in Sc
        @assert all(scenario_inputs[s][:T] .== T) "All scenarios must share T"
        @assert all(scenario_inputs[s][:S] .== S) "All scenarios must share S"
        @assert all(scenario_inputs[s][:Z] .== Z) "All scenarios must share Z"
        @assert scenario_inputs[s][:hours_per_period] == hours_per_period "All scenarios must share hrs/period"
    end

    # Collect scenario-specific parameters (demand, variability, sample weights, NSE)
    demand_s        = [scenario_inputs[s][:demand]         for s in Sc] # Matrix T x Z
    variability_s   = [scenario_inputs[s][:variability]    for s in Sc] # Matrix T x G
    sample_weight_s = [scenario_inputs[s][:sample_weight]  for s in Sc] # Vector length T
    nse_cost_s      = [scenario_inputs[s][:nse].NSE_Cost   for s in Sc] # Vector over S
    nse_max_s       = [scenario_inputs[s][:nse].NSE_Max    for s in Sc] # Vector over S


    # Planning (shared) variables
    @variables(m, begin
        vCAP[g in G]             >= 0
        vRET_CAP[g in OLD]       >= 0
        vNEW_CAP[g in NEW]       >= 0

        vE_CAP[g in STOR]        >= 0
        vRET_E_CAP[g in intersect(STOR, OLD)] >= 0
        vNEW_E_CAP[g in intersect(STOR, NEW)] >= 0

        vT_CAP[l in L]           >= 0
        vRET_T_CAP[l in L]       >= 0
        vNEW_T_CAP[l in L]       >= 0
    end)

    # Upper bounds for new builds
    for g in NEW[generators[NEW, :Max_Cap_MW] .> 0]
        set_upper_bound(vNEW_CAP[g], generators.Max_Cap_MW[g])
    end
    for l in L
        set_upper_bound(vNEW_T_CAP[l], lines.Line_Max_Reinforcement_MW[l])
    end

    # Planning capacity equalities
    @constraints(m, begin
        cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]
        cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]

        cCapEnergyOld[g in intersect(STOR, OLD)],
            vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]
        cCapEnergyNew[g in intersect(STOR, NEW)],
            vE_CAP[g] == vNEW_E_CAP[g]

        cTransCap[l in L],
            vT_CAP[l] == lines.Line_Max_Flow_MW[l] - vRET_T_CAP[l] + vNEW_T_CAP[l]
    end)

    # Fixed costs
    @expression(m, eFixedCostsGeneration,
        sum(generators.Fixed_OM_cost_per_MWyr[g] * vCAP[g] for g in G) +
        sum(generators.Inv_cost_per_MWyr[g] * vNEW_CAP[g] for g in NEW)
    )
    @expression(m, eFixedCostsStorage,
        sum(generators.Fixed_OM_cost_per_MWhyr[g] * vE_CAP[g] for g in STOR) +
        sum(generators.Inv_cost_per_MWhyr[g] * vNEW_E_CAP[g]
            for g in intersect(STOR, NEW))
    )
    @expression(m, eFixedCostsTransmission,
        sum(lines.Line_Fixed_Cost_per_MW_yr[l] * vT_CAP[l] +
            lines.Line_Reinforcement_Cost_per_MW_yr[l] * vNEW_T_CAP[l] for l in L)
    )

    # Operational variables (indexed by scenario)
    @variables(m, begin
            vGEN[    s in Sc, t in T, g in G           ] >= 0
            vCHARGE[ s in Sc, t in T, g in STOR        ] >= 0
            vSOC[    s in Sc, t in T, g in STOR        ] >= 0
            vNSE[    s in Sc, t in T, seg in S, z in Z ] >= 0
            vFLOW[   s in Sc, t in T, l in L           ]
    end)

    # Demand balance
    @constraint(m, cDemandBalance[s in Sc, t in T, z in Z],
        sum(vGEN[s, t, g] for g in intersect(   generators[generators.zone .== z, :R_ID], G)) +
        sum(vNSE[s, t, seg, z] for seg in S) -
        sum(vCHARGE[s, t, g] for g in intersect(generators[generators.zone .== z, :R_ID], STOR)) -
        demand_s[s][t, z] -
        sum(lines[l, Symbol("z" * string(z))] * vFLOW[s, t, l] for l in L) == 0
    )

    # Operational constraints
    @constraints(m, begin
        cMaxPower[s in Sc, t in T, g in G],
            vGEN[s, t, g] <= variability_s[s][t, g] * vCAP[g]
        cMaxCharge[s in Sc, t in T, g in STOR],
            vCHARGE[s, t, g] <= vCAP[g]
        cMaxSOC[s in Sc, t in T, g in STOR],
            vSOC[s, t, g] <= vE_CAP[g]
        cMaxNSE[s in Sc, t in T, seg in S, z in Z],
            vNSE[s, t, seg, z] <= nse_max_s[s][seg] * demand_s[s][t, z]
        cMaxFlow[s in Sc, t in T, l in L],  vFLOW[s, t, l] <= vT_CAP[l]
        cMinFlow[s in Sc, t in T, l in L],  vFLOW[s, t, l] >= -vT_CAP[l]
    end)

    # Ramping and SOC with wrap
    STARTS = minimum(T):hours_per_period:maximum(T)
    INTERIORS = setdiff(T, STARTS)

    @constraints(m, begin
        # Ramp-up
        cRampUp[s in Sc, t in INTERIORS, g in G],
            vGEN[s, t, g] - vGEN[s, t - 1, g] <=
                generators.Ramp_Up_percentage[g] * vCAP[g]
        cRampUpWrap[s in Sc, t in STARTS, g in G],
            vGEN[s, t, g] - vGEN[s, t + hours_per_period - 1, g] <=
                generators.Ramp_Up_percentage[g] * vCAP[g]

        # Ramp-down (including wrap) — fixed
        cRampDown[s in Sc, t in INTERIORS, g in G],
            vGEN[s, t - 1, g] - vGEN[s, t, g] <=
                generators.Ramp_Dn_percentage[g] * vCAP[g]
        cRampDownWrap[s in Sc, t in STARTS, g in G],
            vGEN[s, t + hours_per_period - 1, g] - vGEN[s, t, g] <=
                generators.Ramp_Dn_percentage[g] * vCAP[g]

        # SOC dynamics and wrap
        cSOC[s in Sc, t in INTERIORS, g in STOR],
            vSOC[s, t, g] == vSOC[s, t - 1, g] +
                generators.Eff_up[g] * vCHARGE[s, t, g] -
                vGEN[s, t, g] / generators.Eff_down[g]
        cSOCWrap[s in Sc, t in STARTS, g in STOR],
            vSOC[s, t, g] == vSOC[s, t + hours_per_period - 1, g] +
                generators.Eff_up[g] * vCHARGE[s, t, g] -
                vGEN[s, t, g] / generators.Eff_down[g]
    end)

    # Expected operational costs -- *Weighted* for stochastic optimization
    @expression(
        m,
        eOperationCosts,
        sum(
            scenario_weights[s] * (
                sum(
                    sample_weight_s[s][t] * generators.Var_Cost[g] * vGEN[s, t, g]
                    for t in T, g in G
                ) +
                sum(
                    sample_weight_s[s][t] * nse_cost_s[s][seg] *
                    vNSE[s, t, seg, z]
                    for t in T, seg in S, z in Z
                )
            ) for s in Sc
        )
    )

    @objective(
        m,
        Min,
        eFixedCostsGeneration + eFixedCostsStorage + eFixedCostsTransmission +
        eOperationCosts
    )

    if silent
        set_silent(m)
    end
    return m
end

## INPUTS PATH

In [ ]:
# Read input data for a case with 10 sample days of data
inputs_path = "../../complex_expansion_data/10_days/"

In [ ]:
base_inputs = load_inputs(inputs_path)

println("Number of timesteps: $(length(base_inputs[:T]))")
println("Number of generators: $(nrow(base_inputs[:generators]))")
println("Number of zones: $(length(base_inputs[:zones]))")

# Optional --
# The default storage isn't called very much at all (especially on this short time scale)
# Artificially decrease its cost so it's used more
base_inputs[:generators][base_inputs[:STOR], "Inv_cost_per_MWhyr"]      /= 1000;
base_inputs[:generators][base_inputs[:STOR], "Fixed_OM_cost_per_MWhyr"] /= 1000;
base_inputs[:generators][base_inputs[:STOR], "Eff_up"]   .= 1
base_inputs[:generators][base_inputs[:STOR], "Eff_down"] .= 1;

In [ ]:
# Set up scenarios

In [ ]:
# demand multipliers, gas multipliers 
scenario_multipliers = [
    (0.90, 0.60),
    (0.95, 0.80),
    (1.00, 1.00),
    (1.05, 1.20),
    (1.10, 1.50),
]
SCENARIOS = []

for (demand_mult, gas_mult) in scenario_multipliers
    this_scenario = deepcopy(base_inputs)
    this_scenario[:demand] .*= demand_mult



    # Not important -- reverse the VarOM cost to be able to update the gas price in-situ
    thermal_G = this_scenario[:generators].Heat_rate_MMBTU_per_MWh .!= 0
    derived_Cost_per_MMBtu = (this_scenario[:generators].Var_Cost[thermal_G] .-
            this_scenario[:generators].Var_OM_cost_per_MWh[thermal_G]
      ) ./ this_scenario[:generators].Heat_rate_MMBTU_per_MWh[thermal_G]
    derived_Cost_per_MMBtu *= gas_mult * 5
    this_scenario[:generators].Var_Cost[thermal_G] .= 
                this_scenario[:generators].Var_OM_cost_per_MWh[thermal_G] .+
                derived_Cost_per_MMBtu .* this_scenario[:generators].Heat_rate_MMBTU_per_MWh[thermal_G]
    this_scenario[:generators].Start_Cost[thermal_G] .= 
                this_scenario[:generators].Start_cost_per_MW[thermal_G] .+
                derived_Cost_per_MMBtu .* this_scenario[:generators].Start_fuel_MMBTU_per_MW[thermal_G]
   
    push!(SCENARIOS, this_scenario)

end

# An Example of 3 Equal-Weight Scenarios

In [ ]:

# Create three variants (stochastic scenarios)
inputs_scenarios = [
    SCENARIOS[1]
    SCENARIOS[3]
    SCENARIOS[5]
]

# If want to give arbitrary weights and re-normalize
weights = [1/3, 1/3, 1/3]
weights = weights/sum(weights)

println("Scenario Weights: $(weights)")


# This is a deep truth about computers -- they represent numbers as so-called
# floating point numbers. How do you take the infinite real numbers to
# manageable byte length?
# A classic example is typing (0.1 + 0.2) == 0.3 in almost any programming
# language.
@assert isapprox(sum(weights), 1) "Sum of weights equals 1"
@assert length(weights) == length(inputs_scenarios)

# Build and solve model
stochastic_Model = generate_stochastic_model(base_inputs, inputs_scenarios, weights)
t = @elapsed optimize!(stochastic_Model)

println("======"); println("======"); println("======"); println("======");

println("Expected system cost: ", objective_value(stochastic_Model))
println("Took (seconds): ", t)

# Plotting
Will be slow the first time is run

In [ ]:
using CairoMakie
using DataFrames

# --- 1. Data Preparation (Same as before, still correct) ---
gen = base_inputs[:generators]
G, NEW = base_inputs[:G], base_inputs[:NEW]

zones_g   = gen.zone[G]
types_g   = String.(gen.Resource[G])
final_cap = [value(stochastic_Model[:vCAP][g]) for g in G]
new_cap   = [g in NEW ? value(stochastic_Model[:vNEW_CAP][g]) : 0.0 for g in G]
exist_kept = final_cap .- new_cap

unique_zones = sort(unique(zones_g))
unique_types = sort(unique(types_g))
zlabels = string.("Z", unique_zones)

exist_dict = Dict((z, r) => 0.0 for z in unique_zones, r in unique_types)
new_dict   = Dict((z, r) => 0.0 for z in unique_zones, r in unique_types)
for i in eachindex(G)
    if zones_g[i] in unique_zones
        exist_dict[(zones_g[i], types_g[i])] += exist_kept[i]
        new_dict[(zones_g[i], types_g[i])]   += new_cap[i]
    end
end

plot_df = DataFrame(zone_str=String[], category=String[], resource=String[], capacity=Float64[])
for z in unique_zones
    for r in unique_types
        push!(plot_df, (string("Z", z), "Existing", r, exist_dict[(z, r)]))
        push!(plot_df, (string("Z", z), "New", r, new_dict[(z, r)]))
    end
end
# filter!(row -> row.capacity > 1e-6, plot_df)
sort!(plot_df, "capacity"; rev=false)

zone_to_int = Dict(label => i for (i, label) in enumerate(zlabels))
category_to_int = Dict("Existing" => 1, "New" => 2)
resource_to_int = Dict(rtype => i for (i, rtype) in enumerate(unique_types))

plot_df.zone_int =     [zone_to_int[z] for z in plot_df.zone_str]
plot_df.category_int = [category_to_int[c] for c in plot_df.category]
plot_df.resource_int = [resource_to_int[r] for r in plot_df.resource]
# -------------------------------------------------------------------------

# --- 3. Generate the Plot with Makie using Integer Codes ---
set_theme!(theme_light())
fig = Figure(size = (1200, 700))

ax = Axis(
    fig[1, 1],
    title = "Generation Capacity by Zone (Existing vs New)",
    xlabel = "Zone (Existing then New Resources)",
    ylabel = "MW",
    xticks = (1:length(zlabels), zlabels)
)


cmap_name = :seaborn_colorblind
colors = Makie.resample_cmap(cmap_name, length(unique_types))

barplot!(
    ax,
    plot_df.zone_int,
    plot_df.capacity,
    dodge = plot_df.category_int,
    # dodge = plot_df.resource_int,
    stack = plot_df.resource_int,
    color = plot_df.resource_int,
    colormap = colors
)

legend_elements = [PolyElement(color = colors[i], strokecolor = :transparent) for i in 1:length(unique_types)]

Legend(
    fig[1, 2],
    legend_elements,
    unique_types, # The string labels for the legend
    "Resource Type"
)

fig

In [ ]:
using CairoMakie
using DataFrames

gen = base_inputs[:generators]
lines = base_inputs[:lines]
G, STOR, L = base_inputs[:G], base_inputs[:STOR], base_inputs[:L]
NEW, Zset = base_inputs[:NEW], base_inputs[:Z]
zlabels = string.("Z", Zset)
zone_to_int = Dict(label => i for (i, label) in enumerate(zlabels))

# --- 2. Create the Figure and Theme ---
set_theme!(theme_light())
p2_exists = !isempty(STOR)
fig = Figure(size = 5.0 .* (p2_exists ? (1200, 800) : (1200, 450)))

# --- Plot 1: Generation Capacity (Stacked) ---
begin
    final_cap = [value(stochastic_Model[:vCAP][g]) for g in G]
    new_cap = [g in NEW ? value(stochastic_Model[:vNEW_CAP][g]) : 0.0 for g in G]
    exist_kept = final_cap .- new_cap
    gen_exist_by_zone = [sum(exist_kept[i] for i in eachindex(G) if gen.zone[G][i] == z) for z in Zset]
    gen_new_by_zone   = [sum(new_cap[i]    for i in eachindex(G) if gen.zone[G][i] == z) for z in Zset]

    df_gen = vcat(
        DataFrame(zone = zlabels, category = "Existing kept", capacity = gen_exist_by_zone),
        DataFrame(zone = zlabels, category = "New", capacity = gen_new_by_zone)
    )
    df_gen.zone_int = [zone_to_int[z] for z in df_gen.zone]
    # Create integer mapping for the stacking variable
    category_to_int_gen = Dict("Existing kept" => 1, "New" => 2)
    df_gen.category_int = [category_to_int_gen[c] for c in df_gen.category]

    ax1 = Axis(fig[1, 1], title="Generation capacity by zone (MW)", xlabel="Zone", ylabel="MW", xticks=(1:length(zlabels), zlabels))
    barplot!(ax1, df_gen.zone_int, df_gen.capacity, stack=df_gen.category_int, color=df_gen.category_int, colormap=[:steelblue, :orange])
    Legend(fig[1, 1, BottomRight()], [PolyElement(color=c) for c in [:steelblue, :orange]], ["Existing kept", "New"])
end

# --- Plot 2: Storage Energy (Stacked) ---
if p2_exists
    final_e = [value(stochastic_Model[:vE_CAP][g]) for g in STOR]
    new_e = [g in intersect(STOR, NEW) ? value(stochastic_Model[:vNEW_E_CAP][g]) : 0.0 for g in STOR]
    exist_kept_e = final_e .- new_e
    stor_exist_by_zone = [sum(exist_kept_e[i] for i in eachindex(STOR) if gen.zone[STOR][i] == z) for z in Zset]
    stor_new_by_zone   = [sum(new_e[i]        for i in eachindex(STOR) if gen.zone[STOR][i] == z) for z in Zset]

    df_stor = vcat(
        DataFrame(zone = zlabels, category = "Existing kept", capacity = stor_exist_by_zone),
        DataFrame(zone = zlabels, category = "New", capacity = stor_new_by_zone)
    )
    df_stor.zone_int = [zone_to_int[z] for z in df_stor.zone]
    # Create integer mapping for the stacking variable
    category_to_int_stor = Dict("Existing kept" => 1, "New" => 2)
    df_stor.category_int = [category_to_int_stor[c] for c in df_stor.category]

    ax2 = Axis(fig[1, 2], title="Storage energy by zone (MWh)", xlabel="Zone", ylabel="MWh", xticks=(1:length(zlabels), zlabels))
    barplot!(ax2, df_stor.zone_int, df_stor.capacity, stack=df_stor.category_int, color=df_stor.category_int, colormap=[:seagreen, :gold])
    Legend(fig[1, 2, BottomRight()], [PolyElement(color=c) for c in [:seagreen, :gold]], ["Existing kept", "New"])
end

# --- Plot 3: Transmission Capacity (Dodged/Grouped) ---
begin
    init_t = lines.Line_Max_Flow_MW[L]
    final_t = [value(stochastic_Model[:vT_CAP][l]) for l in L]
    line_labels = string.(L)
    line_to_int = Dict(label => i for (i, label) in enumerate(line_labels))

    df_trans = vcat(
        DataFrame(line = line_labels, category = "Initial", capacity = init_t),
        DataFrame(line = line_labels, category = "Final", capacity = final_t)
    )
    df_trans.line_int = [line_to_int[l] for l in df_trans.line]
    # Create integer mapping for the dodging variable
    category_to_int_trans = Dict("Initial" => 1, "Final" => 2)
    df_trans.category_int = [category_to_int_trans[c] for c in df_trans.category]

    ax3_pos = p2_exists ? fig[2, 1] : fig[1, 2]
    ax3 = Axis(ax3_pos, title="Transmission capacity per line (MW)", xlabel="Line ID", ylabel="MW", xticks=(1:length(line_labels), line_labels))
    barplot!(ax3, df_trans.line_int, df_trans.capacity, dodge=df_trans.category_int, color=df_trans.category_int, colormap=[:gray, :steelblue])
    Legend(ax3_pos[1, 1, BottomRight()], [PolyElement(color=c) for c in [:gray, :steelblue]], ["Initial", "Final"])
end

# --- Plot 4: Cost Breakdown (Simple Bar) ---
begin
    cost_labels = ["Fixed Gen", "Fixed Storage", "Fixed Trans", "Expected Ops"]
    cost_values = [
        value(stochastic_Model[:eFixedCostsGeneration]),
        value(stochastic_Model[:eFixedCostsStorage]),
        value(stochastic_Model[:eFixedCostsTransmission]),
        value(stochastic_Model[:eOperationCosts]),
    ]

    ax4_pos = p2_exists ? fig[2, 2] : fig[1, 3]
    ax4 = Axis(ax4_pos, title="Cost breakdown", xlabel="Component", ylabel="Cost",
        xticks=(1:length(cost_labels), cost_labels), xticklabelrotation=pi/8)
    barplot!(ax4, 1:length(cost_labels), cost_values, color=:mediumpurple)
end

fig